# USDA Food Nutrient Classifier

## Introduction

## Notebook setup

In [ ]:
import os
import re
import nltk
import sklrean
import pandas as pd
# ...

## Dataset

src: USDA

## Import dataset

In [ ]:
data = pd.read_json(path_or_buf="./data/...")

## Standardize units

## Training classifier